<a href="https://colab.research.google.com/github/Bohdan-at-Kulinich/Machine-Learning/blob/main/Convnets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Introduction to Convolutional Neural Networks

In [1]:
# Instantiating a small convnet: 
# using the Functional API

from tensorflow import keras 
from tensorflow.keras import layers 

# a convnet takes as input tensors of shape (image_height, image_width, image_channels) not including the batch dimension: 
# (28, 28, 1) is the format of MNIST images. 
inputs = keras.Input(shape=(28, 28, 1)) 

x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs) 
x = layers.MaxPooling2D(pool_size=2)(x) 
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x) 
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x) 
x = layers.Flatten()(x) 

outputs = layers.Dense(10, activation='softmax')(x) 

model = keras.Model(inputs=inputs, outputs=outputs) 

In [ ]:
# Displaying the model's summary: 

model.summary() 

# The output of each Conv2D and MaxPooling2D layer is a rank-3 tensor (height, width, channels). 
# The heigth and width dimensions tend to shrink as we go deeper into the model. 
# The number of channels is cotrolled by the first argument passed to the Conv2D layers (32, 64, 128). 

# After the last Conv2D layer we end up with an output of shape (3, 3, 128): 
# a 3x3 feature map of 128 channels. 
# In the next step we need to feed this rank-3 output into the a densely connected calssifier:
# a stack of Dense layers, which process 1D vectors. 
# We need to flatten the 3D outputs to 1D with a Flatten layer befor adding the Dense layers. 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 3, 128)         73856 

In [ ]:
# Training the convnet on MNIST dataset: 

# Since we are doing 10-way classification with a softmax output, 
# we use categorical crossentropy loss, 
# and because our labels are integers, 
# we use the sparse version, sparse_categorical_crossentropy: 

from tensorflow.keras.datasets import mnist 

(train_images, train_labels), (test_images, test_labels) = mnist.load_data() 

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255 

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255 

model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels,
          epochs=5, 
          batch_size=64) 

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 36s 38ms/step - loss: 0.1587 - accuracy: 0.9501
Epoch 2/5
938/938 [==============================] - 36s 39ms/step - loss: 0.0452 - accuracy: 0.9862
Epoch 3/5
938/938 [==============================] - 35s 38ms/step - loss: 0.0315 - accuracy: 0.9904
Epoch 4/5
938/938 [==============================] - 36s 38ms/step - loss: 0.0229 - accuracy: 0.9927
Epoch 5/5
938/938 [==============================] - 36s 38ms/step - loss: 0.0186 - accuracy: 0.9940


In [ ]:
# Evaluating the convnet: 

test_loss, test_acc = model.evaluate(test_images, test_labels) 
print(f"Test accuracy: {test_acc:.3f}") 

313/313 [==============================] - 2s 7ms/step - loss: 0.0376 - accuracy: 0.9881
Test accuracy: 0.988


### The convolution operation: 

Dense layers learn global patterns in their input feature space (for MNIST 
digit, patterns involving app pixels). 

Convolution layers learn local patterns (for images, patterns found in small
2D windows of the inputs): 

1.   Patterns are translation invariant: a certain pattern learned in the one corner of the image, is recognized by convnet anywhere.
2.   Convnets can learn spatial heirarchies of patterns. This allows to learn increasingly complex and abstract visual concepts, based on the previously learned small local patterns (such as edges, elementary lines, and textures). 

Convolutions operate over rank-3 tensors called *feature maps*, with two spacial axes (height, width) as well as *depth* axis (*channels* axis). For an RGB image, the dimension of the depth axis is 3, since the image has 3 color channels: red, green and blue. For a black-and-white picture, the depth is 1 (levels of gray). 

The convolution operation pruduces an output feature map which is still an rank-3 tensor. Its depth can be arbitrary, because the output depth is a parameter of the layer. The different channels now stand for *filters*.  

Filters encode specific aspects of the input data (like "presence of a face in the input" at a high-level). 

Convolutions are defined by two key parameters: 

1.   Size of the patches extracted from the inputs: typically 3x3 or 5x5.
2.   Depth of the input feature map: number of filters computed by the convolution. 

in Keras: Conv2D(output_depth, (window_height, window_width)). 





In [2]:
# An incorrectly structured convnet missing its max-pooling layers: 

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x) 
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x) 
x = layers.Flatten()(x) 
outputs = layers.Dense(10, activation='softmax')(x) 

model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs) 

# This model is not conductive to learning a spatial heirarchy of features: 
# the high-level patterns learned by the convnet are still very small with regard to the initial input.
# There are too many total coefficients which will lead to overfitting. 

In [3]:
# Medel summary: 
model_no_max_pool.summary() 

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 22, 22, 128)       73856     
                                                                 
 flatten_1 (Flatten)         (None, 61952)             0         
                                                                 
 dense_1 (Dense)             (None, 10)                619530    
                                                                 
Total params: 712,202
Trainable params: 712,202
Non-trainab

### Training a convnet from scratch on a small dataset: 

In [9]:
# Downloading the data: 
# ( Dogs vs. Cats dataset from Kaggle)

from google.colab import files  

files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

!kaggle competitions download -c dogs-vs-cats 

!unzip -qq train.zip 


Saving kaggle (2).json to kaggle (2) (1).json
mkdir: cannot create directory ‘/root/.kaggle’: File exists
401 - Unauthorized
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [11]:
# Copying images to training, validation, and test directories: 

import os, shutil, pathlib 

original_dir = pathlib.Path('train')     # path to the directory where the original dataset was uncompressed
new_base_dir = pathlib.Path('cats_vs_dogs_small')  # directory where we will store our smaller dataset 

def make_subset(subset_name, start_index, end_index): 
  for category in ('cat', 'dog'):
    dir = new_base_dir / subset_name / category 
    os.makedirs(dir) 
    fname = [f"{category}.{i}.jpg"
             for i in range(start_index, end_index)]
    for name in fname: 
      shutil.copyfile(src=original_dir / fname, 
                      dst=dir / fname) 
      
make_subset('train', start_index=0, end_index=1000) 
make_subset('validation', start_index = 1000, end_index=1500) 
make_subset('test', start_index=1500, end_index=2500) 


FileExistsError: ignored